We already used some graph metrics in the previous tutorial. There we will cover graphs metrics and features in details. Also, we will cover Brain Connectivity Toolbox usage.

## 1. Realworld dataset

Here we use UCLA autism dataset publicly available at the UCLA Multimodal Connectivity Database. Data includes DTI-based connectivity matrices of 51 high-functioning ASD subjects (6 females) and 43 TD subjects (7 females).

In [1]:
from reskit.datasets import load_UCLA_data


X, y = load_UCLA_data()
X = X['matrices']

## 2. Normalizations and Graph Metrics

We can normalize and build some metrics.

In [2]:
from reskit.normalizations import mean_norm
from reskit.features import bag_of_edges
from reskit.core import MatrixTransformerf


normalized_X = MatrixTransformer(
    func=mean_norm).fit_transform(X)

featured_X = MatrixTransformer(
    func=bag_of_edges).fit_transform(normalized_X)

## 4. Brain Connectivity Toolbox

We provide some basic graph metrics in Reskit. To access most state of the art graph metrics you can use Brain Connectivity Toolbox. You should install it via pip:

Let's calculate pagerank centrality of a random graph using BCT python library.

In [3]:
from bct.algorithms.centrality import pagerank_centrality
import numpy as np


pagerank_centrality(np.random.rand(3,3), d=0.85)

array([ 0.49519608,  0.35932666,  0.14547726])

Now we calculates this metric for UCLA dataset. d is the pagerank_centrality parameter, called damping factor (see bctpy documentation for more info). 

In [4]:
featured_X = MatrixTransformer(
    d=0.85,
    func=pagerank_centrality).fit_transform(X)

If we want to try pagerank_centrality and degrees for SVM and LogisticRegression classfiers.

In [5]:
from bct.algorithms.degree import degrees_und

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from reskit.core import Pipeliner

# Feature extraction step variants (1st step)
featurizers = [('pagerank', MatrixTransformer(    
                                d=0.85,
                                func=pagerank_centrality)),
               ('degrees', MatrixTransformer(
                                func=degrees_und))]

# Models (3rd step)
classifiers = [('LR', LogisticRegression()),
               ('SVC', SVC())]

# Reskit needs to define steps in this manner
steps = [('featurizer', featurizers),
         ('classifier', classifiers)]

# Grid search parameters for our models
param_grid = {'LR': {'penalty': ['l1', 'l2']},
              'SVC': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}}

# Quality metric that we want to optimize
scoring='roc_auc'

pipe = Pipeliner(steps, param_grid=param_grid)
pipe.plan_table

,featurizer,classifier
0,pagerank,LR
1,pagerank,SVC
2,degrees,LR
3,degrees,SVC


In [6]:
pipe.get_results(X, y, scoring=scoring, caching_steps=['featurizer'])

Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,featurizer,classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,pagerank,LR,0.5,0,{'penalty': 'l1'},0.5,0,[ 0.5 0.5 0.5]
1,pagerank,SVC,0.523565,0.049125,{'kernel': 'rbf'},0.523249,0.0492934,[ 0.55294118 0.56302521 0.45378151]
2,degrees,LR,0.5346,0.0167932,{'penalty': 'l2'},0.53436,0.016723,[ 0.55686275 0.51680672 0.52941176]
3,degrees,SVC,0.552512,0.00940143,{'kernel': 'poly'},0.552381,0.00936597,[ 0.56470588 0.55042017 0.54201681]


This is the main things about maching learning on graphs. Now you can try big amount of normalizations features and classifiers for graphs classifcation. In case you need something specific you can implement temporary pipeline step to fiegure out the influence of this step on the result.